In [1]:
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
type(iris)

sklearn.utils._bunch.Bunch

In [3]:
dir(iris)

['DESCR',
 'data',
 'data_module',
 'feature_names',
 'filename',
 'frame',
 'target',
 'target_names']

In [5]:
import pandas as pd
flowers_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
X = flowers_df

In [6]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [7]:
y = pd.DataFrame(data = iris.target, columns=['target'])

In [8]:
y

,target
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7, random_state=5)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [12]:
def get_score(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [13]:
get_score(LogisticRegression(), X_train, X_test, y_train, y_test)

C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9777777777777777

In [14]:
get_score(SVC(), X_train, X_test, y_train, y_test)

C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9777777777777777

In [15]:
get_score(RandomForestClassifier(), X_train, X_test, y_train, y_test)

C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.9333333333333333

When n_splits = 4 in k-fold cross-validation, it means the dataset will be divided into 4 folds. In each iteration of the cross-validation loop, one fold will be used as the test set, and the remaining folds will be combined to form the training set.

For each iteration:

    3 out of 4 folds will be used for training, which corresponds to 75% of the data.
    1 out of 4 folds will be used for testing, which corresponds to 25% of the data.

This process will be repeated 4 times, with each fold serving as the test set once. This allows every data point to be used for both training and testing across different iterations, providing a more robust estimate of the model's performance compared to a single train-test split

hence all the given data will be used as a testingset once 
and all the given data will be used as a trainingset multiple times

#SV#
KFold: when you have 100 data points and n_splits as 4, then
    we will have four iteration.
    in 1st iteration, first 25 rows as test data & remaining 75 rows as training data
    in 2nd iteration, next 25 rows as test data(25:50) & remaining 75 rows as training data
    in 3rd iteration, next 25 rows as test data(50:75) & remaining 75 rows as training data
    in 4th iteration, next 25 rows as test data(75:100) & remaining 75 rows as training data
we get the average prediction from the above results

#corrections from chat GPT#
So, while the test set will indeed consist of 25 data points in each iteration, they won't necessarily be the first 25 rows, next 25 rows, and so on. Instead, they'll be randomly selected from the shuffled dataset.

we get the average prediction from the above results --> this is wrong.Your understanding about using majority voting for prediction across iterations is not typical in KFold cross-validation. In each iteration, you train a model on the training set and evaluate its performance on the test set. You don't combine predictions from multiple iterations but rather assess the performance of the model in each individual iteration to get an overall estimate of its performance.

KFold cross-validation is quite good! It's a robust technique for estimating the performance of a machine learning model.

In KFold cross-validation, the data is divided into k folds of equal size, where each fold is used as the test set exactly once, and the remaining k-1 folds are used as the training set.
KFold does not take into account the distribution of the target variable. As a result, it may lead to suboptimal performance, especially in cases where the classes are imbalanced. For example, if one class is significantly more prevalent than the others, there's a chance that some folds might not contain any instances of the minority class, leading to biased evaluation

KFold is a general-purpose cross-validation method, StratifiedKFold is specifically designed for classification tasks, particularly when dealing with imbalanced datasets, to ensure that each fold is representative of the overall class distribution

In [20]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

scores_logistic = []
scores_svm = []
scores_rf = []

for train_index, test_index in folds.split(X,iris.target):
    print(len(train_index))
    print(len(test_index))
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], \
                                       y.iloc[train_index], y.iloc[test_index]
    scores_logistic.append(get_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X_train, X_test, y_train, y_test))  
    scores_svm.append(get_score(SVC(gamma='auto'), X_train, X_test, y_train, y_test))
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=40), X_train, X_test, y_train, y_test))

120
30
120
30
120
30


C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (

120
30
120
30


C:\Python_Vignesh\jnb_venv\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [21]:
scores_logistic ,scores_svm,scores_rf

([1.0, 0.9666666666666667, 0.9333333333333333, 0.9, 1.0],
 [0.9666666666666667, 1.0, 0.9666666666666667, 0.9666666666666667, 1.0],
 [0.9666666666666667,
  0.9666666666666667,
  0.9333333333333333,
  0.9666666666666667,
  1.0])